In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from HuCapa_trainer import evaluateFromList
from Ecapa_trainer import EcapaTrainer
from HuCapa_trainer import HuCapaTrainer
from neural_compressor.training import prepare_compression
from neural_compressor.config import DistillationConfig, KnowledgeDistillationLossConfig
from models.HuCapa import HuCapa
from models.CustomEcapa import ECAPA_TDNN
import torch
import time
from models.RawNet3 import RawNet3
from models.ResNetSE34L import ResNetSE
from models.RawNetBasicBlock import Bottle2neck
from models.ResNetBlocks import SEBasicBlock
from tuneThreshold import *

/home/iref/Repos/voxceleb_trainer/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [2]:
from DatasetLoader import loadWAV

In [3]:
%cd ..

/home/iref/Repos/voxceleb_trainer


# Prepare Vox-like test list

In [3]:
frame = pd.read_csv("./data/protocol.csv")

In [4]:
frame

,enroll,verify,enroll_user,verify_user,predict_class
0,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0001.wav,121,121,1
1,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0002.wav,121,121,1
2,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0003.wav,121,121,1
3,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0004.wav,121,121,1
4,test-clean/121/121726/121-121726-0000.wav,test-clean/121/121726/121-121726-0005.wav,121,121,1
...,...,...,...,...,...
1721026,test-clean/908/31957/908-31957-0022.wav,test-clean/908/31957/908-31957-0024.wav,908,908,1
1721027,test-clean/908/31957/908-31957-0022.wav,test-clean/908/31957/908-31957-0025.wav,908,908,1
1721028,test-clean/908/31957/908-31957-0023.wav,test-clean/908/31957/908-31957-0024.wav,908,908,1
1721029,test-clean/908/31957/908-31957-0023.wav,test-clean/908/31957/908-31957-0025.wav,908,908,1


In [15]:
t = frame.apply(lambda row: str(row["predict_class"]) + " " + row["enroll"] + " " +  row["verify"], axis=1)

In [21]:
with open("./data/ls_test_list.txt", "w+") as f:
    for line in t.to_list():
        f.write(f"{line}\n") 

# Evaluation

In [4]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [5]:
ecp = EcapaTrainer(test_path="./data/librispeech/", test_file="./data/ls_test_list.txt", batch_size=256, device=device)

Initialised AAMSoftmax margin 0.200 scale 30.000
06-08 18:57:59 Overall parameters: = 6978176.00
Learnable model parameters: 6978176
Learnable loss parameters: 1150848
Sched type: StepLR


In [6]:
print(f"Learnable model parameters: {sum(p.numel() for p in ecp.model.parameters())}")

Learnable model parameters: 6978176


In [7]:
ecp.load_params("./exps/Ecapa/custom_ver/1-10/1-10_cp.tar")

10

In [7]:
sample = loadWAV("./data/librispeech/test-clean/121/121726/121-121726-0001.wav", max_frames=200)

In [8]:
sample = torch.FloatTensor(sample)#.cuda()

In [10]:
%%timeit
ecp.model(sample)

58.1 ms ± 2.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
ecp.eval(10)

Reading 2600 of 2620: 132.44 Hz, embedding size 192
Computing 1721000 of 1721031: 6556.34 Hz
 2023-05-25 01:44:55 Epoch 10, VEER 2.5881, MinDCF 0.12464


(2.58805363143072, 0.12463600442646094)

In [11]:
hcp = HuCapaTrainer(test_path="./data/librispeech/", test_file="./data/ls_test_list.txt", batch_size=256, max_frames = 200, device=device)

Initialised AAMSoftmax margin 0.200 scale 30.000
06-08 18:33:03 Overall parameters: = 118661837.00
Learnable parameters: 24290125
Sched type: StepLR


In [12]:
print(f"Learnable model parameters: {sum(p.numel() for p in hcp.model.parameters())}")

Learnable model parameters: 118661837


In [13]:
hcp.load_params("./exps/HuCapa/cyclic_sched_2/16-24/16-24_cp.tar")

24

In [14]:
%%timeit
hcp.model(sample)

780 ms ± 46.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
hcp.eval()

Reading 2600 of 2620: 24.00 Hz, embedding size 192
Computing 1721000 of 1721031: 6726.16 Hz
 2023-03-13 16:53:34 VEER 2.1498, MinDCF 0.11260


(2.149824382075005, 0.11260072423964769)

In [3]:
teacher = HuCapa(device=device)
student = ECAPA_TDNN(512)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill/0-20_distill_ecapa_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-05-10 14:52:24 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 14:52:24 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 14:52:24 [INFO] student_targets_loss: CE, 0.5
2023-05-10 14:52:24 [INFO] teacher_student_loss: MSE, 0.5


FileNotFoundError: [Errno 2] No such file or directory: 'exps/distill/0-20_distill_ecapa_cp.tar'

In [17]:
def eval(model):
    model.eval()
    sc, lab, _ = evaluateFromList("./data/ls_test_list.txt", "./data/librispeech/", model, 16)
    #sc, lab, _ = evaluateFromList("./data/test_list.txt", "./data/voxceleb1/", model, 16)
    result = tuneThresholdfromScore(sc, lab, [1, 0.1])
    fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
    mindcf, _ = ComputeMinDcf(fnrs, fprs, thresholds, p_target=0.05, c_miss=1, c_fa=1)
    print('\n',time.strftime("%Y-%m-%d %H:%M:%S"), "Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}".format(20, result[1], mindcf))
    return result[1], mindcf

In [18]:
eval(model)

Reading 2600 of 2620: 123.02 Hz, embedding size 192
Computing 1721000 of 1721031: 6694.94 Hz
 2023-03-13 17:03:08 Epoch 20, VEER 2.6642, MinDCF 0.13470


(2.664172855884561, 0.13469824807435213)

In [3]:
test = Path("./exps/HuCapa/cyclic_sched_2/13-15/")

In [6]:
test.parent.stem

'cyclic_sched_2'

In [9]:
from SpeakerNet import SpeakerNet, ModelTrainer, WrappedModel

# RawNet3

In [10]:
rawnet = SpeakerNet("RawNet3", "adam", "aamsoftmax", 1, nOut=192, encoder_type="ECA", sinc_stride=10, nClasses=5994, margin=0.1, scale=30)
rawnet = WrappedModel(rawnet).to(device)

self.encoder_type ECA
Initialised AAMSoftmax margin 0.100 scale 30.000


In [11]:
trainer = ModelTrainer(rawnet, "adam", "steplr", 0, False, lr=0.001, weight_decay=5e-05, test_interval=2, max_epoch=20, lr_decay=0.75)

Initialised Adam optimizer
Initialised step LR scheduler


In [12]:
print(f"Learnable model parameters: {sum(p.numel() for p in trainer.__model__.parameters() if p.requires_grad)}")

Learnable model parameters: 17234370


In [13]:
trainer.loadParameters("exps/RawNet3_AAM_150/model/model000000020.model")

In [15]:
%%timeit
trainer.__model__(sample)

476 ms ± 14.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
eval(trainer.__model__)

Reading 2600 of 2620: 33.64 Hz, embedding size 192
Computing 1721000 of 1721031: 6510.46 Hz
 2023-05-10 15:45:00 Epoch 20, VEER 1.8489, MinDCF 0.09137


(1.8489076589710693, 0.09136965527282837)

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher = HuCapa(device=device)
student = RawNet3(
            Bottle2neck, 
            model_scale=8, 
            context=True, 
            summed=True, 
            out_bn=False, 
            log_sinc=True, 
            norm_sinc="mean", 
            grad_mult=1, 
            nOut=192,
            encoder_type="ECA",
            sinc_stride=10
        )
student.to(device)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill/rawnet_batch_150/10-29/11-30_rawnet_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-05-10 15:52:01 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 15:52:01 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 15:52:01 [INFO] student_targets_loss: CE, 0.5
2023-05-10 15:52:01 [INFO] teacher_student_loss: MSE, 0.5


self.encoder_type ECA


PyTorchFXModel(
  (_model): RawNet3(
    (preprocess): Sequential(
      (0): PreEmphasis()
      (1): InstanceNorm1d(1, eps=0.0001, momentum=0.1, affine=True, track_running_stats=False)
    )
    (conv1): Encoder(
      (filterbank): ParamSincFB()
    )
    (relu): ReLU()
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Bottle2neck(
      (conv1): Conv1d(256, 1024, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), 

In [39]:
eval(student)

Reading 2600 of 2620: 33.64 Hz, embedding size 192
Computing 1721000 of 1721031: 6822.09 Hz
 2023-05-10 15:57:44 Epoch 20, VEER 2.4695, MinDCF 0.13203


(2.469525124781158, 0.13202612131825048)

# ResNet34

In [16]:
resnet = SpeakerNet("ResNetSE34L", "adam", "amsoftmax", 1, nOut=192, encoder_type="SAP", sinc_stride=10, nClasses=5994, margin=0.3, scale=30, n_mels=40,
log_input=False)
resnet = WrappedModel(resnet).to(device)

Embedding size is 192, encoder SAP.
Initialised AMSoftmax m=0.300 s=30.000


In [17]:
trainer = ModelTrainer(resnet, "adam", "steplr", 0, True, lr=0.001, weight_decay=5e-05, test_interval=2, max_epoch=20, lr_decay=0.75)

Initialised Adam optimizer
Initialised step LR scheduler


In [18]:
print(f"Learnable model parameters: {sum(p.numel() for p in trainer.__model__.parameters() if p.requires_grad)}")

Learnable model parameters: 2546646


In [19]:
trainer.loadParameters("exps/Resnet_amsm_try_2/model/model000000016.model")

In [21]:
%%timeit
trainer.__model__(sample)

23.5 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
trainer.__model__.eval()

WrappedModel(
  (module): SpeakerNet(
    (__S__): ResNetSE(
      (conv1): Conv2d(1, 16, kernel_size=(7, 7), stride=(2, 1), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): SEBasicBlock(
          (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (se): SELayer(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (fc): Sequential(
              (0): Linear(in_features=16, out_features=2, bias=True)
              (1): ReLU(inplace=True)
              (2): Lin

In [46]:
eval(trainer.__model__)

Reading 2600 of 2620: 146.55 Hz, embedding size 192
Computing 1721000 of 1721031: 6995.11 Hz
 2023-05-18 01:09:29 Epoch 20, VEER 5.6734, MinDCF 0.28642


(5.673421031631545, 0.2864215741587345)

In [15]:
teacher = HuCapa(device=device)
num_filters = [16, 32, 64, 128]
student =  ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, 192)
student.to(device)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill/resnet_450_am/0-19/1-20_resnet_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-05-10 16:37:31 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 16:37:31 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 16:37:31 [INFO] student_targets_loss: CE, 0.5
2023-05-10 16:37:31 [INFO] teacher_student_loss: MSE, 0.5


Embedding size is 192, encoder SAP.


PyTorchFXModel(
  (_model): ResNetSE(
    (conv1): Conv2d(1, 16, kernel_size=(7, 7), stride=(2, 1), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): SEBasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (fc): Sequential(
            (0): Linear(in_features=16, out_features=2, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=2, out_features=16, bias=True)
        

In [18]:
eval(student)

Reading 2600 of 2620: 139.51 Hz, embedding size 192
Computing 1721000 of 1721031: 6505.61 Hz
 2023-05-10 16:43:30 Epoch 20, VEER 4.8700, MinDCF 0.24916


(4.869956478236049, 0.24916380261211332)

# MSE ONLY ECAPA

In [20]:
teacher = HuCapa(device=device)
student = ECAPA_TDNN(512)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill_mse_only/0-20_distill_ecapa_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-05-10 17:29:51 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 17:29:51 [WARNING] Force convert framework model to neural_compressor model.
2023-05-10 17:29:51 [INFO] student_targets_loss: CE, 0.5
2023-05-10 17:29:51 [INFO] teacher_student_loss: MSE, 0.5


PyTorchFXModel(
  (_model): ECAPA_TDNN(
    (torchfbank): Sequential(
      (0): PreEmphasis()
      (1): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
    )
    (conv1): Conv1d(80, 512, kernel_size=(5,), stride=(1,), padding=(2,))
    (relu): ReLU()
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Bottle2neck(
      (conv1): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (4): Conv1d(64, 64, kernel_size=(3,), s

In [21]:
eval(student)

Reading 2600 of 2620: 120.39 Hz, embedding size 192
Computing 1721000 of 1721031: 6614.19 Hz
 2023-05-10 17:34:42 Epoch 20, VEER 2.7316, MinDCF 0.14087


(2.7315927404008233, 0.14086584788404094)

# REDUCED ECAPA

In [22]:
teacher = HuCapa(device=device)
student = ECAPA_TDNN(128)
distillation_criterion = KnowledgeDistillationLossConfig(loss_types=["CE", "MSE"], loss_weights=[0.5, 0.5])
conf = DistillationConfig(teacher_model=teacher, criterion=distillation_criterion, optimizer={"SGD": {"learning_rate": 0.0001}})
compression_manager = prepare_compression(student, conf)
compression_manager.callbacks.on_train_begin()
model = compression_manager.model
checkpoint = torch.load("exps/distill_reduced_ecapa_128/0-20_distill_ecapa_cp.tar")
model.load_state_dict(checkpoint["ecapa"])
model.to(device)

2023-06-08 18:59:45 [WARNING] Force convert framework model to neural_compressor model.
2023-06-08 18:59:45 [WARNING] Force convert framework model to neural_compressor model.
2023-06-08 18:59:45 [INFO] student_targets_loss: CE, 0.5
2023-06-08 18:59:45 [INFO] teacher_student_loss: MSE, 0.5


PyTorchFXModel(
  (_model): ECAPA_TDNN(
    (torchfbank): Sequential(
      (0): PreEmphasis()
      (1): MelSpectrogram(
        (spectrogram): Spectrogram()
        (mel_scale): MelScale()
      )
    )
    (conv1): Conv1d(80, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (relu): ReLU()
    (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Bottle2neck(
      (conv1): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (convs): ModuleList(
        (0): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (1): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (3): Conv1d(16, 16, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (4): Conv1d(16, 16, kernel_size=(3,), s

In [23]:
print(f"Learnable model parameters: {sum(p.numel() for p in student.parameters() if p.requires_grad)}")

Learnable model parameters: 3031472


In [24]:
%%timeit
student(sample)

33.4 ms ± 688 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
eval(student)

Reading 2600 of 2620: 140.04 Hz, embedding size 192
Computing 1721000 of 1721031: 6580.41 Hz
 2023-05-10 17:42:48 Epoch 20, VEER 3.1655, MinDCF 0.17360


(3.165472319787732, 0.17359648353724624)